# Laboratorio 5

1. Utilice los archivos pre-procesados que obtuvo en la práctica 3, mismos que almacenan la colección CACM y los queries.
2. Si en la práctica tres generó el vocabulario sin ordenarlo alfabéticamente ordénelo, tanto el normal como el reducido.
3. Represente cada documento empleando el modelo vectorial y el esquema de pesado tf. Recuerde que la dimensión de los vectores será igual al tamaño del vocabulario que obtuvo en la práctica 4. Trabajen con el vocabulario sin reducir y después reducido. Guarda las matrices generada, que representa los documentos de la colección, para utilizarla en prácticas posteriores.
4. Utiliza el archivo de queries y represéntelos también empleando el modelo vectorial y esquema de pesado tf. Con vocabulario normal y reducido. Guarda las matrices generadas.

Primero extraemos las palabras de documentos_final.txt


In [1]:
filename = "documentos_trunc.txt"
file = open(filename, 'rt',encoding='utf-8-sig')
text = file.read()
file.close()
import re
words = re.split(r"\n", text)
doc_words = ""
doc_count = 0
for w in words:
    split = re.split(r"\|", w)
    if len(split) == 1:
        continue
    elif len(split) == 4:
        #doc_data[i]['d_num'] = split[0]
        doc_words += split[1] + " " # titulo
        doc_words += split[2] + " " # texto
    elif len(split) == 3 or len(split) == 2:
        doc_words += split[1] + " " # titulo
    doc_count += 1
doc_words = [x for x in (doc_words).split(" ") if x]
print(doc_words[:10])
print(doc_count)

['preliminari', 'report', 'intern', 'algebra', 'languag', 'extract', 'root', 'repeat', 'subtract', 'digit']
3204


Luego, importamos el vocabulario creado en la práctica 4.

In [2]:
filename = "vocabulario_trunc.txt"
file = open(filename, 'rt',encoding='utf-8-sig')
vocabulary = file.read().split(", ")
file.close()
vocabulary = sorted(vocabulary)
print(len(vocabulary))

5178


Ahora, comenzamos los cálculos para la representación del vocabulario por medio del modelo vectorial.

In [5]:
# Primero, obtenemos las frecuencias de las palabras

from collections import Counter
from operator import countOf
dict_freq = dict().fromkeys(vocabulary)
dict_freq = { x : 0 for x in dict_freq}
#for v in vocabulary:
#    dict_freq[v] = countOf(doc_words, v)
#dict_freq = dict(sorted(Counter(doc_words).items()))
for w in doc_words:
    dict_freq[w] = dict_freq[w] + 1
print(len(dict_freq.items()))
#print(c.items())
dict_freq['preliminari']

5178


16

In [28]:
# separando las palabras por documento:
# text
from itertools import count
from operator import countOf


words = re.split(r"\n", text)
doc_words_per_doc = dict()
i = 1
for w in words:
    split = re.split(r"\|", w)
    index = str(i)
    if len(split) == 1:
        continue
    elif len(split) == 4:
        doc_words_per_doc[i] = [x for x in split[2].split(" ") if x]
    elif len(split) == 3:
        doc_words_per_doc[i] = [x for x in split[1].split(" ") if x]
    i += 1
doc_words_per_doc[1]

# contamos las frecuencias de cada término por documento
freq_per_doc = dict()
for d in doc_words_per_doc.keys():
    freq_per_doc[d] = dict().fromkeys(dict_freq.keys())
    for t in dict_freq.keys():
        freq_per_doc[d][t] = countOf(doc_words_per_doc[d], t)
        #print(t)
        
   
#{1: {"premiliniari": 1, "report": 3, ... , "algorithm": 0, ...}} -> formato del diccionario
# dict_freq         -> vector q
# freq_per_doc[1]   -> vector d

Ahora lo mismo pero con el vocabulario reducido

In [29]:
import copy
filename = "vocabulario_reducido_trunc.txt"
file = open(filename, 'rt',encoding='utf-8-sig')
red_voc = file.read().split(", ")
file.close()
red_voc = sorted(red_voc)

new_voc = copy.copy(dict_freq)
#freq_per_doc_red = { key : value for (key, value) in dict_freq if key in red_voc }
del_keys = [ key for key in new_voc if key not in red_voc ]
for key in del_keys:
     del new_voc[key]
print(len(dict_freq))
print(len(new_voc))


5178
4670


In [30]:
freq_per_doc_red = dict()
for d in doc_words_per_doc.keys():
    freq_per_doc_red[d] = dict().fromkeys(new_voc.keys())
    for t in new_voc.keys():
        freq_per_doc_red[d][t] = countOf(doc_words_per_doc[d], t)
        
# new_voc           -> vector q reducido
# freq_per_doc_red  -> matriz d reducido

## Con queries
### Vocabulario normal

In [31]:
filename = "querys_trunc.txt"
file = open(filename, 'rt',encoding='utf-8-sig')
text = file.read()
file.close()
import re
words = re.split(r"\n", text)
quer_words_per_quer = dict()
i = 1
for w in words:
    split = re.split(r"\|", w)
    index = str(i)
    if len(split) == 1:
        continue
    else:
        quer_words_per_quer[i] = [x for x in split[1].split(" ") if x]
    i += 1
print(len(quer_words_per_quer))

64


In [32]:
freq_per_quer = dict()
for d in quer_words_per_quer.keys():
    freq_per_quer[d] = dict().fromkeys(dict_freq.keys())
    for t in dict_freq.keys():
        freq_per_quer[d][t] = countOf(quer_words_per_quer[d], t)
        #print(t)

# dict_freq         -> vector q
# freq_per_quer     -> vector d

Vocabulario reducido

In [33]:
freq_per_quer_red = dict()
for d in quer_words_per_quer.keys():
    freq_per_quer_red[d] = dict().fromkeys(new_voc.keys())
    for t in new_voc.keys():
        freq_per_quer_red[d][t] = countOf(quer_words_per_quer[d], t)


# new_voc         -> vector q
# freq_per_quer_red     -> vector d

In [34]:
print(freq_per_quer[1]["tss"])
#print(freq_per_quer_red[1]["tss"])

1


## Con tf-idf

Calcularemos primero un vector con los valores de frecuencia inversa del doc (idf)

In [35]:
#primero, calculamos N_t (# de docs con el término)
from math import log


N_t = dict().fromkeys(dict_freq.keys())
N_t = { x : 0 for x in N_t}
for i in N_t.keys():
    for j in freq_per_doc.keys():
        if freq_per_doc[j][i] != 0:
            N_t[i] += 1


In [40]:
# calculando idf_i
idf_i = dict().fromkeys(N_t.keys())
for i in idf_i.keys():
    idf_i[i] = 0
    if N_t[i]:
        idf_i[i] = log(doc_count / N_t[i]) + 1

idf_i['algorithm']

1.9089829173416073

Obteniendo las matrices

In [41]:
dict_freq_2 = dict().fromkeys(dict_freq.keys())
for i in dict_freq.keys():
    dict_freq_2[i] = dict_freq[i] * idf_i[i]
    
freq_per_doc_2 = dict()
for i in freq_per_doc.keys():
    freq_per_doc_2[i] = dict().fromkeys(dict_freq.keys())
    for j in dict_freq.keys():
        freq_per_doc_2[i][j] = freq_per_doc[i][j] * idf_i[j]
        
# dict_freq_2         -> vector q
# freq_per_doc_2   -> vector d

In [42]:
new_voc_2 = dict().fromkeys(new_voc.keys())
for i in new_voc_2.keys():
    new_voc_2[i] = new_voc[i] * idf_i[i]
    
freq_per_doc_red_2 = dict()
for i in freq_per_doc_red.keys():
    freq_per_doc_red_2[i] = dict().fromkeys(new_voc.keys())
    for j in new_voc.keys():
        freq_per_doc_red_2[i][j] = freq_per_doc_red[i][j] * idf_i[j]
        
# new_voc_2           -> vector q reducido
# freq_per_doc_red_2  -> matriz d reducido

## Con queries

In [43]:
 
freq_per_quer_2 = dict()
for i in freq_per_quer.keys():
    freq_per_quer_2[i] = dict().fromkeys(dict_freq.keys())
    for j in dict_freq.keys():
        freq_per_quer_2[i][j] = freq_per_quer[i][j] * idf_i[j]
        
# dict_freq_2         -> vector q
# freq_per_quer_2     -> vector d

In [44]:
freq_per_quer_red_2 = dict()
for i in freq_per_quer_red.keys():
    freq_per_quer_red_2[i] = dict().fromkeys(new_voc.keys())
    for j in new_voc.keys():
        freq_per_quer_red_2[i][j] = freq_per_quer_red[i][j] * idf_i[j]

# new_voc_2         -> vector q
# freq_per_quer_red_2     -> vector d

### Guardando los resultados a 8 archivos distintos:


In [50]:
# 1 vocabulario normal, documentos
# dict_freq         -> vector q
# freq_per_doc[1]   -> vector d
import json

#with open('Resultados\\doc_q_normal.txt', 'w') as f:
#    f.write(json.dumps(dict_freq))
#    f.close()
with open('Resultados\\doc_d_normal.txt', 'w') as f:
    f.write(json.dumps(freq_per_doc))
    f.close()

In [51]:
# 2. vocabulario reducido, documentos
#with open('Resultados\\doc_q_red.txt', 'w') as f:
#    f.write(json.dumps(new_voc))
#    f.close()
with open('Resultados\\doc_d_red.txt', 'w') as f:
    f.write(json.dumps(freq_per_doc_red))
    f.close()

In [53]:
# 3. vocabulario normal, queries
with open('Resultados\\quer_d_normal.txt', 'w') as f:
    f.write(json.dumps(freq_per_quer))
    f.close()

In [52]:
# 4. vocabulario reducido, queries
with open('Resultados\\quer_d_red.txt', 'w') as f:
    f.write(json.dumps(freq_per_quer_red))
    f.close()

In [54]:
# 5. voc normal, docs, tf-idf
with open('Resultados\\doc_d_normal-2.txt', 'w') as f:
    f.write(json.dumps(freq_per_doc_2))
    f.close()

In [55]:
# 6. voc red, docs, tf-idf
with open('Resultados\\doc_d_red-2.txt', 'w') as f:
    f.write(json.dumps(freq_per_doc_red_2))
    f.close()

In [56]:
# 7. vocabulario normal, queries, tf-idf
with open('Resultados\\quer_d_normal-2.txt', 'w') as f:
    f.write(json.dumps(freq_per_quer_2))
    f.close()

In [57]:
# 8. vocabulario reducido, queries, tf-idf
with open('Resultados\\quer_d_red-2.txt', 'w') as f:
    f.write(json.dumps(freq_per_quer_red_2))
    f.close()